In [5]:
import torch
import torch.nn as nn
from torchvision import models
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import os
from pathlib import Path
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [8]:
base_dir = r"C:\Users\user\Desktop\DL"
test_dir  = os.path.join(base_dir, "test")

# Validation and Test transforms for no randomness, cvonsistent evaluation
val_test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_dataset  = datasets.ImageFolder(test_dir, transform=val_test_transforms)
class_names = test_dataset.classes
num_classes = len(class_names)  # your class_names list

batch_size = 32 # num_workers=2 for parallel data loading
loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

In [9]:
def load_model(model_name, num_classes, checkpoint_path):
    if model_name == "resnet50":
        model = models.resnet50(weights=None)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
    elif model_name == "efficientnet_b0":
        model = models.efficientnet_b0(weights=None)
        num_ftrs = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ftrs, num_classes)
    elif model_name == "densenet121":
        model = models.densenet121(weights=None)
        num_ftrs = model.classifier.in_features
        model.classifier = nn.Linear(num_ftrs, num_classes)
    elif model_name == "mobilenet_v2":
        model = models.mobilenet_v2(weights=None)
        num_ftrs = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ftrs, num_classes)
    else:
        raise ValueError(f"Model '{model_name}' not supported")

    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model.to(device)
    model.eval()
    return model

In [10]:
def evaluate_model(model, loader, class_names):
    y_true, y_pred = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="macro")
    print("\nClassification Report:\n")
    print(classification_report(y_true, y_pred, target_names=class_names))

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f"Confusion Matrix - {model.__class__.__name__}")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.tight_layout()
    plt.show()
    
    return acc, f1

In [ ]:
models_checkpoints = {
    "resnet50": "ResNet50_Finetuned_best.pth",
    "efficientnet_b0": "EfficientNetB0_best.pth",
    "densenet121": "DenseNet121_best.pth",
    "mobilenet_v2": "MobileNetV2_best.pth"
}

In [ ]:
results = {}

for model_name, ckpt in models_checkpoints.items():
    print(f"\nEvaluating {model_name}...")
    model = load_model(model_name, num_classes, ckpt)
    acc, f1 = evaluate_model(model, loader, class_names)
    results[model_name] = {"accuracy": acc, "f1": f1}

names = list(results.keys())
accs = [results[m]["accuracy"] for m in names]
f1s  = [results[m]["f1"] for m in names]

x = np.arange(len(names))
width = 0.35

plt.figure(figsize=(10,5))
plt.bar(x - width/2, accs, width, label="Accuracy", alpha=0.7)
plt.bar(x + width/2, f1s, width, label="F1 Score", alpha=0.7)
plt.xticks(x, names)
plt.ylabel("Score")
plt.title("Model Comparison")
plt.ylim(0,1)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
print("\nFinal Results Comparison:")
for model_name in results:
    print(f"{model_name}: Accuracy={results[model_name]['accuracy']:.4f}, F1={results[model_name]['f1']:.4f}")